In [ ]:
pip install yfinance

In [ ]:
pip install ta

In [ ]:
pip install scikit-learn

In [ ]:
import yfinance as yf
from ta import add_all_ta_features
from ta.utils import dropna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def fetch_stock_data(ticker, start_date, end_date):
    try:
        # Fetch historical stock data using yfinance
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        print("Error fetching data:", e)
        return None

In [ ]:
def perform_technical_analysis(stock_data):
    try:
        # Reset index
        stock_data.reset_index(inplace=True)

        # Add technical analysis features
        stock_data = dropna(stock_data)
        stock_data = add_all_ta_features(stock_data, open="Open", high="High", low="Low", close="Close", volume="Volume")

        return stock_data
    except Exception as e:
        print("Error performing technical analysis:", e)
        return None


In [ ]:
def train_model(X, y):
    # Split features and target into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a machine learning model
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Model Accuracy:", accuracy)

    return model, X_train, X_test, y_train, y_test

In [ ]:
def predict_movement(model, features):
    # Predict stock movement
    prediction = model.predict([features])

    return prediction

In [ ]:
def main():
    # Input stock ticker
    ticker = input("Enter the stock ticker: ")
    print("Fetching data for ticker:", ticker)  # Debugging statement

    # Fetch historical stock data
    start_date = input("Enter start date (YYYY-MM-DD): ")  # Change start date as needed
    end_date = input("Enter end date (YYYY-MM-DD): ")    # Change end date as needed
    stock_data = fetch_stock_data(ticker, start_date, end_date)

    if stock_data is None:
        print("Failed to fetch stock data for ticker:", ticker)
        return

    print("Fetched data successfully.")  # Debugging statement

    print("Stock data before technical analysis:", stock_data)  # Debugging statement

    # Perform technical analysis
    stock_data = perform_technical_analysis(stock_data)

    if stock_data is None:
        print("Failed to perform technical analysis.")
        return

    print("Performed technical analysis successfully.")  # Debugging statement

    # Prepare features and target
    features = stock_data.drop(['Close', 'Date'], axis=1)
    target = (stock_data['Close'] - stock_data['Open']).apply(lambda x: 1 if x > 0 else 0)

    # Train machine learning model
    model, _, _, _, _ = train_model(features, target)  # Unpack only the model

    # Predict stock movement
    prediction = predict_movement(model, features.iloc[-1])

    # Print prediction
    if prediction[0] == 1:
        print("This stock will rise.")
    else:
        print("This stock will go down.")

if __name__ == "__main__":
    main()